# CP321-ILR: Assigment-9

In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
from utils.helpers import load_data3, plot_trajectories
from utils.models import TPGMM
from utils.utils import *

# TPGMM Implementation

#### theory reference : [1](https://ieeexplore.ieee.org/document/6907339),[2](https://calinon.ch/papers/Calinon-JIST2015.pdf)

In [ ]:
Data, time = load_data3("c")

In [ ]:
As, Bs = plot_trajectories(Data, time)

In [ ]:
mp = TPGMM(As, Bs, 4)
mp.fit(Data)

In [ ]:
# using old params of trajectory 1
A_new, B_new = As[:, 0], Bs[:, 0]
mp.plot_gaussians_wrt_frames(Data, A_new, B_new)

In [ ]:
# translating start and end of the trajectory - 2  by -10
A_new, B_new = As[:, 1].copy(), Bs[:, 1].copy()
B_new[0] = B_new[0] + np.array([0, -10, 0])
B_new[1] = B_new[1] + np.array([0, -10, 0])
mp.plot_gaussians_wrt_frames(Data, A_new, B_new)

## Dataset "C"

### 1. Different start and end points

Try different start and end points for particular trajectory.

#### Plots

### 3. Generalization of the algorithm

Try changing the `n_mixture` parameter and comment on the generalization of the algorithm.

#### Plots

#### Analysis

## Dataset "J"

### 1. Different start and end points

Try different start and end points for particular trajectory.

#### Plots

### 3. Generalization of the algorithm

Try changing the `n_mixture` parameter and comment on the generalization of the algorithm.

#### Plots

#### Analysis

## Dataset "S"

### 1. Different start and end points

Try different start and end points for particular trajectory.

#### Plots

### 3. Generalization of the algorithm

Try changing the `n_mixture` parameter and comment on the generalization of the algorithm.

#### Plots

#### Analysis